# Lab 11 Linear programming in SCIP

More examples of using SCIP

- [Ev Charging Station Optimization](#EV-Charging-Station-Optimization)
- [Disaster Relief Logistics Optimization](#Disaster-Relief-Logistics-Optimization)

In [2]:
from pyscipopt import Model

## EV Charging Station Optimization

### Problem Description

You are managing **three electric vehicle (EV) charging stations**: **Station A, Station B, and Station C**. Your objective is to allocate a limited supply of electricity across these stations in a way that **maximizes the overall net benefit**.

Each station has the following characteristics:

| Station | Max Capacity (kWh) | Revenue per kWh | Time Cost per kWh |
|---------|--------------------|------------------|--------------------|
| A       | 40                 | 5                | 2                  |
| B       | 30                 | 4                | 1.5                |
| C       | 50                 | 6                | 3                  |

You also have a constraint on total energy available:  
**Maximum energy supply: 100 kWh**


### Objective

Maximize the **net benefit**, defined as:

$$
\text{Net Benefit} = \sum_{\text{stations}} (\text{Revenue per kWh} - \text{Time Cost per kWh}) \times \text{Energy Allocated}
$$

More concretely:

$$
\text{Maximize: } (5 - 2) \cdot x_A + (4 - 1.5) \cdot x_B + (6 - 3) \cdot x_C = 3x_A + 2.5x_B + 3x_C
$$


### Decision Variables

Let \( x_A, x_B, x_C \) represent the amount of energy (in kWh) allocated to Station A, B, and C, respectively.


### Constraints

$$
x_A + x_B + x_C \leq 100 \quad \text{(Total energy supply constraint)} \\
0 \leq x_A \leq 40 \\
0 \leq x_B \leq 30 \\
0 \leq x_C \leq 50
$$


###  Goal

Formulate and solve this problem using **PySCIPOpt**, extract the optimal energy allocation per station, and compute the resulting **total net benefit**.


In [3]:
# Parameters
stations = ['A', 'B', 'C']
max_energy = 100  # total kWh available

# For each station: (max_kWh, revenue_per_kWh, time_cost_per_kWh)
station_data = {
    'A': (40, 5, 2),   # Station A: max 40 kWh, $5 revenue/kWh, $2 cost/kWh
    'B': (30, 4, 1.5),
    'C': (50, 6, 3),
}

# Create optimization model
model = Model("EVChargingAllocation")

# Add variables for energy allocated to each station
energy_vars = {}
for s in stations:
    energy_vars[s] = model.addVar(name=f"energy_{s}", lb=0, ub=station_data[s][0])

# Add total energy constraint
model.addCons(sum(energy_vars[s] for s in stations) <= max_energy, "TotalEnergy")

# Set objective: maximize total revenue - total time cost
model.setObjective(
    sum((station_data[s][1] - station_data[s][2]) * energy_vars[s] for s in stations),
    "maximize"
)

# Solve
model.optimize()

# Output solution
print("\n--- Optimal Energy Allocation ---")
for s in stations:
    val = model.getVal(energy_vars[s])
    print(f"Station {s}: {val:.2f} kWh")

print("Total Net Benefit:", model.getObjVal())



--- Optimal Energy Allocation ---feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:

Station A: 40.00 kWh
Station B: 10.00 kWh
Station C: 50.00 kWh
Total Net Benefit: 295.0
(round 1, fast)       2 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       2 del vars, 1 del conss, 0 add conss, 1 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 3 deleted vars, 1 deleted constraints, 0 added constraints, 1 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +2.95000000000000e+02 (2 solutions)
Dual Bound         : +2.9500000

### Optimization Summary

The SCIP solver successfully found the **optimal solution** with **zero gap** between the primal and dual bounds, indicating the solution is globally optimal. The total **net benefit** achieved is:

$$
\text{Net Benefit} = 295.0
$$

### Optimal Energy Allocation

The energy distribution across the three charging stations is as follows:

| Station | Energy Allocated (kWh) |
|---------|------------------------|
| A       | 40.00                  |
| B       | 10.00                  |
| C       | 50.00                  |

- **Station A** is allocated its **maximum capacity** (40 kWh).
- **Station B** is allocated **10 kWh**, well below its capacity (30 kWh), indicating it is **less cost-effective** compared to others.
- **Station C** is also allocated its **maximum capacity** (50 kWh), which suggests it is one of the **most profitable** options in terms of net benefit (revenue – time cost).


## Disaster Relief Logistics Optimization

### Problem Description

In the aftermath of a natural disaster, emergency supplies (such as food, water, and medicine) must be delivered from several central **warehouses** to multiple **disaster relief zones**. Each warehouse has a limited supply of goods, and each relief zone has a minimum demand that must be met.

The goal is to determine how many units of supplies should be shipped from each warehouse to each zone such that:

- All zone demands are met
- Warehouse supplies are not exceeded
- The **total transportation cost is minimized**

This is a classical **linear programming problem** known as the **transportation problem**.


### Data

**Warehouse Supply (units):**

| Warehouse | Supply |
|-----------|--------|
| W1        | 100    |
| W2        | 120    |
| W3        | 150    |

**Zone Demand (units):**

| Zone | Demand |
|------|--------|
| Z1   | 80     |
| Z2   | 70     |
| Z3   | 90     |
| Z4   | 100    |

**Cost per unit shipped (in USD):**

| From \ To | Z1 | Z2 | Z3 | Z4 |
|-----------|----|----|----|----|
| W1        | 4  | 6  | 9  | 8  |
| W2        | 5  | 4  | 7  | 6  |
| W3        | 8  | 7  | 5  | 3  |


### Decision Variables

Let \( x_{ij} \) be the number of units shipped from **warehouse \( i \)** to **zone \( j \)**.


### Objective Function

Minimize the total shipping cost:

$$
\text{Minimize } \sum_{i=1}^{3} \sum_{j=1}^{4} c_{ij} \cdot x_{ij}
$$

Where \( c_{ij} \) is the cost of shipping one unit from warehouse \( i \) to zone \( j \).


### Constraints

1. **Supply constraints** for each warehouse:

$$
\sum_{j=1}^{4} x_{ij} \leq \text{Supply}_i \quad \text{for all } i
$$

2. **Demand constraints** for each zone:

$$
\sum_{i=1}^{3} x_{ij} \geq \text{Demand}_j \quad \text{for all } j
$$

3. **Non-negativity constraints**:

$$
x_{ij} \geq 0 \quad \text{for all } i, j
$$

### Goal

Determine the optimal values of \( x_{ij} \) that minimize total cost while satisfying all supply and demand constraints. Solve the problem using a linear programming solver such as **PySCIPOpt**, **PuLP**, or **Pyomo**.


In [4]:
# Define data
warehouses = ["W1", "W2", "W3"]
zones = ["Z1", "Z2", "Z3", "Z4"]

supply = {"W1": 100, "W2": 120, "W3": 150}
demand = {"Z1": 80, "Z2": 70, "Z3": 90, "Z4": 100}

cost = {
    ("W1", "Z1"): 4, ("W1", "Z2"): 6, ("W1", "Z3"): 9, ("W1", "Z4"): 8,
    ("W2", "Z1"): 5, ("W2", "Z2"): 4, ("W2", "Z3"): 7, ("W2", "Z4"): 6,
    ("W3", "Z1"): 8, ("W3", "Z2"): 7, ("W3", "Z3"): 5, ("W3", "Z4"): 3
}

# Create model
model = Model("DisasterRelief")

# Define decision variables
x = {}
for w in warehouses:
    for z in zones:
        x[(w, z)] = model.addVar(name=f"x_{w}_{z}", lb=0.0)

# Supply constraints
for w in warehouses:
    model.addCons(sum(x[(w, z)] for z in zones) <= supply[w], name=f"supply_{w}")

# Demand constraints
for z in zones:
    model.addCons(sum(x[(w, z)] for w in warehouses) >= demand[z], name=f"demand_{z}")

# Objective: Minimize total cost
model.setObjective(sum(cost[(w, z)] * x[(w, z)] for w in warehouses for z in zones), "minimize")

# Solve
model.optimize()

# Print solution
print("\n--- Optimal Shipping Plan ---")
total_cost = 0
for w in warehouses:
    for z in zones:
        val = model.getVal(x[(w, z)])
        if val > 1e-6:
            print(f"Ship {val:.2f} units from {w} to {z} (Cost: ${cost[(w, z)]})")
            total_cost += cost[(w, z)] * val

print(f"Total Minimum Cost: ${total_cost:.2f}")



--- Optimal Shipping Plan ---
Ship 80.00 units from W1 to Z1 (Cost: $4)
Ship 70.00 units from W2 to Z2 (Cost: $4)
Ship 40.00 units from W2 to Z3 (Cost: $7)
Ship 50.00 units from W3 to Z3 (Cost: $5)
Ship 100.00 units from W3 to Z4 (Cost: $3)
Total Minimum Cost: $1430.00
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 12 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 23 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 23 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 12 variables (0 bin, 0 int, 0 impl, 12 cont) and 7 constr

### Optimization Summary

The SCIP solver has successfully found the **optimal shipping plan** with **zero duality gap**, confirming the solution is **globally optimal**.

- **Total Minimum Cost**: \$1430.00
- **Solver Time**: 0.00 seconds
- **Solution Status**: Optimal

The solver applied **presolving** techniques to simplify the model by tightening bounds and quickly solved the resulting continuous linear program.


### Optimal Shipping Plan

| From Warehouse | To Zone | Units Shipped | Cost per Unit | Total Cost |
|----------------|---------|----------------|----------------|-------------|
| W1             | Z1      | 80.00          | \$4            | \$320.00     |
| W2             | Z2      | 70.00          | \$4            | \$280.00     |
| W2             | Z3      | 40.00          | \$7            | \$280.00     |
| W3             | Z3      | 50.00          | \$5            | \$250.00     |
| W3             | Z4      | 100.00         | \$3            | \$300.00     |

**Total Cost**:  
$$
320 + 280 + 280 + 250 + 300 = \boxed{\$1430.00}
$$


### Insights

- **Warehouse W1** fully supplies Zone Z1 at the lowest available cost.
- **Warehouse W2** splits its supply between Z2 and Z3, efficiently using its limited stock.
- **Warehouse W3** takes on the rest of Z3 and fully supplies Z4, where it has the **lowest cost per unit** (\$3).
